# Totale

##### setup

In [34]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import altair as alt
import os.path
import getpass
import platform
from pandasql import sqldf
import openpyxl

username = getpass.getuser()

print("Username: ", username)
pathPart1 = "/Users/"
pathPart2 = "/Github/SCGProject/Datasets/"
complePath = pathPart1+username+pathPart2

if platform.system() == "Darwin":
    if(username == "marcovinciguerra"):
        dfCambio = pd.read_csv(
            complePath+"/CorrectedDatasets/dfCambio.csv")

        dfClienti = pd.read_csv(
            complePath+"/CorrectedDatasets/dfClienti.csv")

        dfConsumi = pd.read_csv(
             complePath + "/CorrectedDatasets/dfConsumi.csv")

        dfCostoOrarioConsuntivo = pd.read_csv(
            complePath+"/CorrectedDatasets/dfConsuntivo.csv")

        dfCostoOrarioBudget = pd.read_csv(
            complePath+"CorrectedDatasets/dfCostoOrario.csv")

        dfImpiegoRisorse = pd.read_csv(
         complePath+"/CorrectedDatasets/dfImpiegoRisorse.csv")

        dftuttiClienti = pd.read_csv(
         complePath+"/CorrectedDatasets/dftuttiClienti.csv")

        dfVendite = pd.read_csv(
         complePath+"/CorrectedDatasets/dfVendite.csv")

    elif(username == "davidguzman"):
            pathPart1 = "/Users/"
            pathPart2 = "/documents/Github/SCGProject/Datasets/"
            complePath = pathPart1+username+pathPart2

            dfCambio = pd.read_csv(
                complePath+"/CorrectedDatasets/dfCambio.csv")

            dfClienti = pd.read_csv(
                complePath+"/CorrectedDatasets/dfClienti.csv")

            dfConsumi = pd.read_csv(
                complePath + "/CorrectedDatasets/dfConsumi.csv")

            dfCostoOrarioConsuntivo = pd.read_csv(
                complePath+"/CorrectedDatasets/dfConsuntivo.csv")

            dfCostoOrarioBudget = pd.read_csv(
                complePath+"CorrectedDatasets/dfCostoOrario.csv")

            dfImpiegoRisorse = pd.read_csv(
            complePath+"/CorrectedDatasets/dfImpiegoRisorse.csv")

            dftuttiClienti = pd.read_csv(
            complePath+"/CorrectedDatasets/dftuttiClienti.csv")

            dfVendite = pd.read_csv(
            complePath+"/CorrectedDatasets/dfVendite.csv")

elif platform.system() == "Linux":
    complePath = "/home/alinux/unibg/git_repo/SCGProject"
    pathPart1 = "/Users/"
    pathPart2 = "/documents/Github/SCGProject/Datasets/"
    complePath = pathPart1+username+pathPart2

    dfCambio = pd.read_csv(
        complePath+"/CorrectedDatasets/dfCambio.csv")

    dfClienti = pd.read_csv(
        complePath+"/CorrectedDatasets/dfClienti.csv")

    dfConsumi = pd.read_csv(
        complePath + "/CorrectedDatasets/dfConsumi.csv")

    dfCostoOrarioConsuntivo = pd.read_csv(
        complePath+"/CorrectedDatasets/dfConsuntivo.csv")

    dfCostoOrarioBudget = pd.read_csv(
        complePath+"CorrectedDatasets/dfCostoOrario.csv")

    dfImpiegoRisorse = pd.read_csv(
        complePath+"/CorrectedDatasets/dfImpiegoRisorse.csv")

    dftuttiClienti = pd.read_csv(
        complePath+"/CorrectedDatasets/dftuttiClienti.csv")

    dfVendite = pd.read_csv(
        complePath+"/CorrectedDatasets/dfVendite.csv")



dfCambio["TassoCambioMedio"] = dfCambio["TassoCambioMedio"].str.replace(',','.')


Username:  davidguzman


# Ricavi

In [35]:
dfVendite.rename(columns = {'budget/cons':'budget'}, inplace = True)
venditeConsuntivo = sqldf("SELECT DISTINCT * FROM dfVendite WHERE budget = 'CONSUNTIVO' or budget = 'Consuntivo'")
venditeBudget = sqldf("SELECT DISTINCT * FROM dfVendite WHERE budget = 'BUDGET' or budget = 'Budget'")

In [36]:
# join with dfCambio and dfClienti
venditeConsuntivo = sqldf('''SELECT DISTINCT v.NrMovimento, v.budget, v.NrArticolo, v.Quantity, v.ImportoVenditaValutaLocaleTOTALEVENDITA,k.TassoCambioMedio,v.ImportoVenditaValutaLocaleTOTALEVENDITA/k.TassoCambioMedio as TotaleEuro, v.NrOrigine, c.CodCondizioniPagam, c.FattCumulative, c.Valuta
FROM venditeConsuntivo as v join dfClienti as c on v.NrOrigine = c.Nr join dfCambio as k on c.Valuta = k.CodiceValuta
where k.Anno = 'Consuntivo' or k.Anno = 'CONSUNTIVO'
''')

venditeBudget = sqldf('''SELECT DISTINCT v.NrMovimento, v.budget, v.NrArticolo, v.Quantity, v.ImportoVenditaValutaLocaleTOTALEVENDITA,k.TassoCambioMedio,v.ImportoVenditaValutaLocaleTOTALEVENDITA/k.TassoCambioMedio as TotaleEuro, v.NrOrigine, c.CodCondizioniPagam, c.FattCumulative, c.Valuta
FROM venditeBudget as v join dfClienti as c on v.NrOrigine = c.Nr join dfCambio as k on c.Valuta = k.CodiceValuta
where k.Anno = 'Budget' or k.Anno = 'BUDGET'
''')

In [37]:
sumVenditeConsuntivo = sqldf('select sum(TotaleEuro) as sumVenditaTotale from venditeConsuntivo')
sumVenditeBudget = sqldf('select sum(TotaleEuro) as sumVenditaTotale from venditeBudget')
print(sumVenditeConsuntivo)
print(sumVenditeBudget)
print(sumVenditeConsuntivo-sumVenditeBudget) #questo è lo scostamento totale dei ricavi

   sumVenditaTotale
0     507842.851413
   sumVenditaTotale
0     395335.892443
   sumVenditaTotale
0      112506.95897


# Costi

Materie prime

In [38]:
dfConsumi.rename(columns = {'Budget/cons':'budget'}, inplace = True)
consumiConsuntivo = sqldf("SELECT DISTINCT * FROM dfConsumi WHERE budget = 'CONSUNTIVO'")
consumiBudget = sqldf("SELECT DISTINCT * FROM dfConsumi WHERE budget = 'BUDGET'")
sumConsumiConsuntivo = sqldf('select sum(ImportoCostoTOTALE) from consumiConsuntivo')
sumConsumiBudget = sqldf('select sum(ImportoCostoTOTALE) from consumiBudget')
print(sumConsumiConsuntivo)
print(sumConsumiBudget)
print(sumConsumiConsuntivo-sumConsumiBudget) #questo è lo scostamento totale della materia prima

   sum(ImportoCostoTOTALE)
0                 289639.5
   sum(ImportoCostoTOTALE)
0                192906.47
   sum(ImportoCostoTOTALE)
0                 96733.03


Lavorazioni interne

In [39]:
impiegoConsuntivo = sqldf("SELECT DISTINCT * FROM dfImpiegoRisorse WHERE budgetConsuntivo = 'CONSUNTIVO' or budgetConsuntivo = 'Consuntivo'")
print(impiegoConsuntivo.shape)
#join con i costi orari
impiegoConsuntivo  = sqldf('''SELECT DISTINCT i.NrArticolo,i.budgetConsuntivo,i.NrOrdineProduzione,i.Descrizione,i.NrAreaProduzione,i.Risorsa,i.QuantitydiOutput,i.TempoRisorsa,c.CostoOrario
FROM impiegoConsuntivo as i join dfCostoOrarioConsuntivo as c 
on c.AreaProduzione = i.NrAreaProduzione and c.Risorsa = i.Risorsa''')
print(impiegoConsuntivo.shape)
# ci sono alcune quantità di output nulle, per gestire le sommiamo alle tuple con tutte le altre cose uguali
impiegoConsuntivo = sqldf('''select NrArticolo,budgetConsuntivo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,sum(TempoRisorsa) as TempoRisorsa,CostoOrario, sum(QuantitydiOutput) as QuantitydiOutput
from impiegoConsuntivo
group by NrArticolo,budgetConsuntivo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,CostoOrario
order by NrArticolo
''')
impiegoConsuntivo

(3791, 9)
(3723, 9)


,NrArticolo,budgetConsuntivo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,TempoRisorsa,CostoOrario,QuantitydiOutput
0,ART0000018,CONSUNTIVO,ODP19-05998,Controllo qualità,CQ,RIS16,0.0000,15,2.0
1,ART0000018,CONSUNTIVO,ODP19-05998,Fresatura,A30,RIS5,1.0000,50,2.0
2,ART0000018,CONSUNTIVO,ODP19-05998,Tornitura,A20,RIS2,1.0000,80,2.0
3,ART0000041,CONSUNTIVO,ODP19-05998,Controllo qualità,CQ,RIS16,0.0000,15,4.0
4,ART0000041,CONSUNTIVO,ODP19-05998,Fresatura,A30,RIS1,0.0001,80,4.0
...,...,...,...,...,...,...,...,...,...
3012,ART0005136,CONSUNTIVO,ODP19-06532,Fresatura,A30,RIS14,1.2500,50,3.0
3013,ART0005136,CONSUNTIVO,ODP19-06533,Controllo qualità,CQ,RIS16,0.0000,15,4.0
3014,ART0005136,CONSUNTIVO,ODP19-06533,Tornitura,A20,RIS13,0.5000,80,4.0
3015,ART0005136,CONSUNTIVO,ODP19-06688,Controllo qualità,CQ,RIS16,0.0000,15,9.0


In [40]:
impiegoBudget = sqldf("SELECT DISTINCT * FROM dfImpiegoRisorse WHERE budgetConsuntivo = 'BUDGET' or budgetConsuntivo = 'Budget'")
#join con i costi orari
print(impiegoBudget.shape)
impiegoBudget  = sqldf('''SELECT DISTINCT i.NrArticolo,i.budgetConsuntivo,i.NrOrdineProduzione,i.Descrizione,i.NrAreaProduzione,i.Risorsa,i.QuantitydiOutput,i.TempoRisorsa,c.CostoOrario
FROM impiegoBudget as i join dfCostoOrarioBudget as c 
on c.AreaProduzione = i.NrAreaProduzione and c.Risorsa = i.Risorsa''')
print(impiegoBudget.shape)
# righe a consuntivo sono di più che a budget e la somma dei 2 non corrisponde al numero di righe della tabella impiego risorse

# ci sono alcune quantità di output nulle, per gestire le sommiamo alle tuple con tutte le altre cose uguali
impiegoBudget = sqldf('''select NrArticolo,budgetConsuntivo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,sum(TempoRisorsa) as TempoRisorsa,CostoOrario, sum(QuantitydiOutput) as QuantitydiOutput
from impiegoBudget
group by NrArticolo,budgetConsuntivo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,CostoOrario
order by NrArticolo
''')
impiegoBudget


(2473, 9)
(2450, 9)


,NrArticolo,budgetConsuntivo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,TempoRisorsa,CostoOrario,QuantitydiOutput
0,ART0000018,BUDGET,ODP17-05342,Controllo qualità,CQ,RIS16,0.00,12.75,2.0
1,ART0000018,BUDGET,ODP17-05342,Fresatura,A30,RIS7,0.75,51.00,2.0
2,ART0000018,BUDGET,ODP17-05342,Tornitura,A20,RIS2,1.00,68.00,2.0
3,ART0000041,BUDGET,ODP17-05342,Controllo qualità,CQ,RIS16,0.00,12.75,4.0
4,ART0000041,BUDGET,ODP17-05342,Fresatura,A30,RIS7,1.25,51.00,4.0
...,...,...,...,...,...,...,...,...,...
2006,ART0005136,BUDGET,ODP17-15077,Controllo qualità,CQ,RIS16,0.00,12.75,10.0
2007,ART0005136,BUDGET,ODP17-15077,Tornitura,A20,RIS14,1.25,51.00,10.0
2008,ART0005136,BUDGET,ODP17-16230,Brocciatura / stozzatrice,A33,RIS13,0.25,42.50,1.0
2009,ART0005136,BUDGET,ODP17-16230,Controllo qualità,CQ,RIS16,0.00,12.75,1.0


In [41]:
impiegoBudget

sqldf('''select NrArticolo, NrOrdineProduzione, Descrizione, NrAreaProduzione, Risorsa, count(*)
from impiegoBudget 
group by NrArticolo, NrOrdineProduzione, Descrizione, NrAreaProduzione, Risorsa
order by count(*) desc''')

sqldf('''select NrArticolo, Descrizione, NrAreaProduzione, Risorsa, count(*)
from impiegoBudget 
group by NrArticolo, Descrizione, NrAreaProduzione, Risorsa
order by count(*) desc''')

,NrArticolo,Descrizione,NrAreaProduzione,Risorsa,count(*)
0,ART0005136,Controllo qualità,CQ,RIS16,19
1,ART0005136,Tornitura,A20,RIS2,13
2,ART0000795,Controllo qualità,CQ,RIS16,11
3,ART0000762,Controllo qualità,CQ,RIS16,9
4,ART0000782,Controllo qualità,CQ,RIS16,8
...,...,...,...,...,...
1395,ART0005136,Brocciatura / stozzatrice,A33,RIS2,1
1396,ART0005136,Foratura/Maschiatura,A32,RIS9,1
1397,ART0005136,Fresatura,A30,RIS1,1
1398,ART0005136,Fresatura,A30,RIS13,1


# Scostamento totale

In [42]:
sumImpiegoConsuntivo = sqldf(
    'SELECT sum(TempoRisorsa*CostoOrario) FROM impiegoConsuntivo WHERE TempoRisorsa>=0')
sumImpiegoBudget = sqldf(
    'SELECT sum(TempoRisorsa*CostoOrario) FROM impiegoBudget WHERE TempoRisorsa>=0')
print(sumImpiegoConsuntivo)
print(sumImpiegoBudget)
print(sumImpiegoConsuntivo-sumImpiegoBudget)

   sum(TempoRisorsa*CostoOrario)
0                    359238.2962
   sum(TempoRisorsa*CostoOrario)
0                   194521.50346
   sum(TempoRisorsa*CostoOrario)
0                   164716.79274


In [43]:
print(sumVenditeConsuntivo.values[0][0])
print(sumImpiegoConsuntivo.values[0][0]+sumConsumiConsuntivo.values[0][0])
print(sumVenditeConsuntivo.values[0][0]-sumImpiegoConsuntivo.values[0][0]-sumConsumiConsuntivo.values[0][0])
print(sumVenditeBudget.values[0][0])
print(sumImpiegoBudget.values[0][0]+sumConsumiBudget.values[0][0])
print(sumVenditeBudget.values[0][0]-sumImpiegoBudget.values[0][0]-sumConsumiBudget.values[0][0])

507842.8514127266
648877.7962
-141034.94478727342
395335.8924431994
387427.97346000024
7907.918983199197


Senza vincoli su tempi negativi

**non cambia perché per ogni tempo negativo esiste un tempo positivo uguale in modulo che, al sommare i tempi delle stesse attività, si annullano a vicenda**

In [44]:
sumImpiegoConsuntivo = sqldf(
    'SELECT sum(TempoRisorsa*CostoOrario) FROM impiegoConsuntivo')
sumImpiegoBudget = sqldf(
    'SELECT sum(TempoRisorsa*CostoOrario) FROM impiegoBudget')
print(sumImpiegoConsuntivo)
print(sumImpiegoBudget)
print(sumImpiegoConsuntivo-sumImpiegoBudget)

   sum(TempoRisorsa*CostoOrario)
0                    359215.7962
   sum(TempoRisorsa*CostoOrario)
0                   194521.50346
   sum(TempoRisorsa*CostoOrario)
0                   164694.29274


In [45]:
print(sumVenditeConsuntivo.values[0][0])
print(sumImpiegoConsuntivo.values[0][0]+sumConsumiConsuntivo.values[0][0])
print(sumVenditeConsuntivo.values[0][0]-sumImpiegoConsuntivo.values[0][0]-sumConsumiConsuntivo.values[0][0])
print(sumVenditeBudget.values[0][0])
print(sumImpiegoBudget.values[0][0]+sumConsumiBudget.values[0][0])
print(sumVenditeBudget.values[0][0]-sumImpiegoBudget.values[0][0]-sumConsumiBudget.values[0][0])

507842.8514127266
648855.2962
-141012.44478727342
395335.8924431994
387427.97346000024
7907.918983199197


# Socostamento di volume

#### Collapse to something more manageable

In [47]:
# Creazione colonna ImpiegoUnitario
impiegoConsuntivo = sqldf(''' select NrArticolo, budgetConsuntivo, NrOrdineProduzione, Descrizione, NrAreaProduzione, Risorsa, TempoRisorsa, TempoRisorsa/QuantitydiOutput as ImpiegoUnitario, CostoOrario ,QuantitydiOutput
from impiegoConsuntivo''')
impiegoConsuntivo.drop(['NrOrdineProduzione'], axis=1, inplace=True) #Non cambia la quantità di righe!

In [48]:
sqldf('select sum(ImpiegoUnitario*CostoOrario*QuantitydiOutput) from impiegoConsuntivo') 

,sum(ImpiegoUnitario*CostoOrario*QuantitydiOutput)
0,329701.5902


In [17]:
sqldf('select sum(TempoRisorsa*CostoOrario) from impiegoConsuntivo') 

,sum(TempoRisorsa*CostoOrario)
0,359215.7962


Notiamo che sono diversi, e diminiuisce se utilizziamo 'ImpiegoUnitario' questo è dovuto a:

In [22]:
x = sqldf('''select NrArticolo,budgetConsuntivo,Descrizione,NrAreaProduzione,Risorsa,TempoRisorsa,ImpiegoUnitario,CostoOrario,QuantitydiOutput, TempoRisorsa*CostoOrario, ImpiegoUnitario*CostoOrario*QuantitydiOutput
from impiegoConsuntivo''')
#in conclusione, i totali cambiano perché ci sono quantità di output nulle! quindi alcuni impieghi diventano nan
# TODO IDEA PER RISOLVERE: fare avg di CostoOrario e sum TempoRisorsa, QuantitydiOutput in quei casi in cui la descrizione è uguale ma la risorsa è diversa, il totale dovrebbe essere uguale

Avg out the null values

In [49]:
impiegoConsuntivo = sqldf(''' select NrArticolo, budgetConsuntivo, Descrizione, NrAreaProduzione, sum(TempoRisorsa) as TempoRisorsa, avg(CostoOrario) as CostoOrario, sum(QuantitydiOutput) as QuantitydiOutput
from impiegoConsuntivo
group by NrArticolo, budgetConsuntivo, Descrizione, NrAreaProduzione
''')

impiegoConsuntivo = sqldf(''' select NrArticolo, budgetConsuntivo, Descrizione, NrAreaProduzione, TempoRisorsa, TempoRisorsa/QuantitydiOutput as ImpiegoUnitario, CostoOrario ,QuantitydiOutput
from impiegoConsuntivo
''')
impiegoConsuntivo


,NrArticolo,budgetConsuntivo,Descrizione,NrAreaProduzione,TempoRisorsa,ImpiegoUnitario,CostoOrario,QuantitydiOutput
0,ART0000018,CONSUNTIVO,Controllo qualità,CQ,0.0000,0.000000,15.000000,2.0
1,ART0000018,CONSUNTIVO,Fresatura,A30,1.0000,0.500000,50.000000,2.0
2,ART0000018,CONSUNTIVO,Tornitura,A20,1.0000,0.500000,80.000000,2.0
3,ART0000041,CONSUNTIVO,Controllo qualità,CQ,0.0000,0.000000,15.000000,4.0
4,ART0000041,CONSUNTIVO,Fresatura,A30,0.0001,0.000025,80.000000,4.0
...,...,...,...,...,...,...,...,...
1281,ART0005136,CONSUNTIVO,Saldatura,A11,21.8550,0.496705,50.000000,44.0
1282,ART0005136,CONSUNTIVO,Taglio,A10,12.0000,12.000000,60.000000,1.0
1283,ART0005136,CONSUNTIVO,Tornitura,A20,143.9091,0.266993,75.903614,539.0
1284,ART0005136,CONSUNTIVO,Tornitura (chiedere a Massimo),A20,3.7500,3.750000,80.000000,1.0


In [50]:
sqldf('select sum(TempoRisorsa*CostoOrario) from impiegoConsuntivo') # si mantiene quasi uguale!!

,sum(TempoRisorsa*CostoOrario)
0,359425.545873


In [51]:
sqldf('select sum(ImpiegoUnitario*CostoOrario*QuantitydiOutput) from impiegoConsuntivo') 

,sum(ImpiegoUnitario*CostoOrario*QuantitydiOutput)
0,358735.055873


la piccola differenza è dovuta alla restante parte di righe con output nulle e senza altre righe con output non nullo da sommare

In [52]:
sqldf('select * from impiegoConsuntivo where ImpiegoUnitario is null') 

,NrArticolo,budgetConsuntivo,Descrizione,NrAreaProduzione,TempoRisorsa,ImpiegoUnitario,CostoOrario,QuantitydiOutput
0,ART0000151,CONSUNTIVO,Fresatura,A30,3.0000,None,50.0,0.0
1,ART0001789,CONSUNTIVO,Fresatura,A30,4.4990,None,50.0,0.0
2,ART0001872,CONSUNTIVO,Fresatura,A30,0.0001,None,80.0,0.0
3,ART0001873,CONSUNTIVO,Fresatura,A30,0.0001,None,80.0,0.0
4,ART0001939,CONSUNTIVO,Fresatura,A30,0.0001,None,80.0,0.0
5,ART0001942,CONSUNTIVO,Fresatura,A30,0.0001,None,80.0,0.0
6,ART0001945,CONSUNTIVO,Fresatura,A30,0.0001,None,80.0,0.0
7,ART0002152,CONSUNTIVO,Fresatura,A30,4.0000,None,50.0,0.0
8,ART0004146,CONSUNTIVO,Montaggio,A40,1.5000,None,50.0,0.0
9,ART0004357,CONSUNTIVO,Fresatura,A30,0.0100,None,50.0,0.0


#### ripetiamo per budget

In [55]:
impiegoBudget = sqldf(''' select NrArticolo, budgetConsuntivo, Descrizione, NrAreaProduzione, sum(TempoRisorsa) as TempoRisorsa, avg(CostoOrario) as CostoOrario, sum(QuantitydiOutput) as QuantitydiOutput
from impiegoBudget
group by NrArticolo, budgetConsuntivo, Descrizione, NrAreaProduzione
''')

impiegoBudget = sqldf(''' select NrArticolo, budgetConsuntivo, Descrizione, NrAreaProduzione, TempoRisorsa, TempoRisorsa/QuantitydiOutput as ImpiegoUnitario, CostoOrario ,QuantitydiOutput
from impiegoBudget
''')
impiegoBudget

,NrArticolo,budgetConsuntivo,Descrizione,NrAreaProduzione,TempoRisorsa,ImpiegoUnitario,CostoOrario,QuantitydiOutput
0,ART0000018,BUDGET,Controllo qualità,CQ,0.00,0.000000,12.750000,2.0
1,ART0000018,BUDGET,Fresatura,A30,0.75,0.375000,51.000000,2.0
2,ART0000018,BUDGET,Tornitura,A20,1.00,0.500000,68.000000,2.0
3,ART0000041,BUDGET,Controllo qualità,CQ,0.00,0.000000,12.750000,4.0
4,ART0000041,BUDGET,Fresatura,A30,1.25,0.312500,51.000000,4.0
...,...,...,...,...,...,...,...,...
1222,ART0005136,BUDGET,Foratura/Maschiatura,A32,3.00,NaN,42.500000,0.0
1223,ART0005136,BUDGET,Fresatura,A30,21.33,0.888750,47.600000,24.0
1224,ART0005136,BUDGET,Montaggio,A40,8.25,NaN,42.500000,0.0
1225,ART0005136,BUDGET,Taglio spezzoni,A20,4.50,0.155172,68.000000,29.0


In [56]:
sqldf('select sum(TempoRisorsa*CostoOrario) from impiegoBudget') # si mantiene quasi uguale!!

,sum(TempoRisorsa*CostoOrario)
0,194739.149842


In [57]:
sqldf('select sum(ImpiegoUnitario*CostoOrario*QuantitydiOutput) from impiegoBudget') 

,sum(ImpiegoUnitario*CostoOrario*QuantitydiOutput)
0,192493.776242


In [59]:
sqldf('select * from impiegoBudget where ImpiegoUnitario is null') 

,NrArticolo,budgetConsuntivo,Descrizione,NrAreaProduzione,TempoRisorsa,ImpiegoUnitario,CostoOrario,QuantitydiOutput
0,ART0000129,BUDGET,Fresatura,A30,0.0090,None,68.0,0.0
1,ART0000157,BUDGET,Tornitura,A20,15.0000,None,68.0,0.0
2,ART0000645,BUDGET,Fresatura,A30,1.0019,None,68.0,0.0
3,ART0000883,BUDGET,Fresatura,A30,1.0000,None,42.5,0.0
4,ART0001866,BUDGET,Rettifica,A34,10.0000,None,42.5,0.0
5,ART0001877,BUDGET,Fresatura,A30,0.0001,None,68.0,0.0
6,ART0001885,BUDGET,Fresatura,A30,0.0001,None,68.0,0.0
7,ART0001917,BUDGET,Fresatura,A30,0.0090,None,68.0,0.0
8,ART0003780,BUDGET,Fresatura,A30,1.5000,None,51.0,0.0
9,ART0004084,BUDGET,Montaggio,A40,3.1500,None,42.5,0.0


#### Merge/Join

In [63]:
impiegoConsuntivo.drop(['budgetConsuntivo'], axis=1, inplace=True)
impiegoBudget.drop(['budgetConsuntivo'], axis=1, inplace=True)
x = pd.merge(impiegoConsuntivo, impiegoBudget, on=['NrArticolo', 'Descrizione', 'NrAreaProduzione'], how='outer')
x

,NrArticolo,Descrizione,NrAreaProduzione,TempoRisorsa_x,ImpiegoUnitario_x,CostoOrario_x,QuantitydiOutput_x,TempoRisorsa_y,ImpiegoUnitario_y,CostoOrario_y,QuantitydiOutput_y
0,ART0000018,Controllo qualità,CQ,0.0000,0.000000,15.0,2.0,0.00,0.000000,12.75,2.0
1,ART0000018,Fresatura,A30,1.0000,0.500000,50.0,2.0,0.75,0.375000,51.00,2.0
2,ART0000018,Tornitura,A20,1.0000,0.500000,80.0,2.0,1.00,0.500000,68.00,2.0
3,ART0000041,Controllo qualità,CQ,0.0000,0.000000,15.0,4.0,0.00,0.000000,12.75,4.0
4,ART0000041,Fresatura,A30,0.0001,0.000025,80.0,4.0,1.25,0.312500,51.00,4.0
...,...,...,...,...,...,...,...,...,...,...,...
1345,ART0004904,Tornitura,A20,NaN,NaN,NaN,NaN,1.75,0.437500,68.00,4.0
1346,ART0004944,Tornitura,A20,NaN,NaN,NaN,NaN,1.75,0.218750,68.00,8.0
1347,ART0005091,Fresatura,A30,NaN,NaN,NaN,NaN,1.00,0.250000,42.50,4.0
1348,ART0005136,Foratura/Maschiatura,A32,NaN,NaN,NaN,NaN,3.00,NaN,42.50,0.0


In [77]:
c = sqldf('select sum(ImpiegoUnitario_x*CostoOrario_x*QuantitydiOutput_x) from x').iloc[0][0]
b = sqldf('select sum(ImpiegoUnitario_y*CostoOrario_y*QuantitydiOutput_y) from x').iloc[0][0]
print(c)
print(b)
print(c-b)
# non altera i totali!!

358735.05587289936
192493.77624166672
166241.27963123264


In [79]:
# Impiego Standard (volume a consuntivo)
ImpiegoStandard = sqldf('select sum(ImpiegoUnitario_y*CostoOrario_y*QuantitydiOutput_x) from x').iloc[0][0]
# Impiego Effettivo (volume e impiego a consuntivo)
ImpiegoEffettivo=sqldf('select sum(ImpiegoUnitario_x*CostoOrario_y*QuantitydiOutput_x) from x').iloc[0][0]
# Budget Effettivo (tutto a consuntivo)
BudgetEffettivo=sqldf('select sum(ImpiegoUnitario_x*CostoOrario_x*QuantitydiOutput_x) from x').iloc[0][0]
print(b)
print(ImpiegoStandard)
print(ImpiegoEffettivo)
print(BudgetEffettivo)

192493.77624166672
310199.3426471812
281289.91565333324
358735.05587289936


In [82]:
#Scostamenti
ScostamentoVolume = ImpiegoStandard-b
ScostamentoImpiego = ImpiegoEffettivo-ImpiegoStandard
ScostamentoPrezzo = BudgetEffettivo-ImpiegoEffettivo
print(ScostamentoVolume)
print(ScostamentoImpiego)
print(ScostamentoPrezzo)
print(ScostamentoVolume+ScostamentoImpiego+ScostamentoPrezzo) # uguale allo scostamento totale per le lavorazioni!!!ß

117705.56640551446
-28909.42699384794
77445.14021956612
166241.27963123264


### Versione simplificata

In [55]:
impiegoConsuntivo = sqldf(''' select NrArticolo, budgetConsuntivo, NrOrdineProduzione, Descrizione, NrAreaProduzione, Risorsa, TempoRisorsa, CostoOrario, TempoRisorsa*CostoOrario as Totale ,QuantitydiOutput
from impiegoConsuntivo''')
impiegoConsuntivo = sqldf('''select NrArticolo, NrOrdineProduzione, Descrizione ,sum(Totale) as Totale, sum(QuantitydiOutput) as QuantitydiOutput
from impiegoConsuntivo
group by NrArticolo, NrOrdineProduzione, Descrizione
''')
#Tolgo NrOrdineProduzione per semplificare il confronto
impiegoConsuntivo = sqldf('''select NrArticolo, Descrizione , Totale, QuantitydiOutput
from impiegoConsuntivo
''')

In [56]:
sqldf('select sum(Totale) from impiegoConsuntivo') # resta uguale al totale precedente!

,sum(Totale)
0,359215.7962


In [57]:
impiegoBudget = sqldf(''' select NrArticolo, budgetConsuntivo, NrOrdineProduzione, Descrizione, NrAreaProduzione, Risorsa, TempoRisorsa, CostoOrario, TempoRisorsa*CostoOrario as Totale ,QuantitydiOutput
from impiegoBudget''')
impiegoBudget = sqldf('''select NrArticolo, NrOrdineProduzione, Descrizione ,sum(Totale) as Totale, sum(QuantitydiOutput) as QuantitydiOutput
from impiegoBudget
group by NrArticolo, NrOrdineProduzione, Descrizione
''')
#Tolgo NrOrdineProduzione per semplificare il confronto
impiegoBudget = sqldf('''select NrArticolo, Descrizione , Totale, QuantitydiOutput
from impiegoBudget
''')

In [58]:
sqldf('select sum(Totale) from impiegoBudget') # resta uguale al totale precedente!

,sum(Totale)
0,194521.50346


In [61]:
# Consuntivo diventa x e budget diventa y
x = pd.merge(impiegoConsuntivo,impiegoBudget, on = ['NrArticolo','Descrizione'], how = 'outer')